In [1]:
from dotenv import load_dotenv
load_dotenv()

True


# FAISS

In [10]:
from langchain.storage import LocalFileStore
from langchain_ollama import OllamaEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

embeddings = OllamaEmbeddings(model="bge-m3")

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [11]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="bge-m3")

DB_INDEX = "LANGCHAIN_FAISS_DB_INDEX"

faiss_db = FAISS.load_local(
    DB_INDEX, cached_embedder, allow_dangerous_deserialization=True)

print(faiss_db.similarity_search("self-rag")[0].page_content)

- (법적·정책적 요건) 국제인도법을 포함한 국제법과 국내법을 준수해야 하며, 인간의 개입 없이 AI가 자동으로 목표를 설정해 공격하는 자율살상무기(LAWS)의 연구개발을 금지
- (기술적 요건) AI 시스템 활용 시 인간의 참여와 통제를 가능하게 하고, 편향을 방지하고 검증 가능성과 투명성, 신뢰성을 보장하며, 오작동과 심각한 장애 발생 위험을 완화하는 체계를 마련해 안전성을 확보
- n (실시 사항) AI 무기의 연구개발 시 △AI 무기의 분류 △법적·정책적 심사 △기술적 심사의 3단계 심사를 거쳐 사업을 추진하며, 법적·정책적 심사와 기술적 심사는 고위험 무기에만 진행
- (AI 무기의 분류) 자율살상무기의 관점에서 집중적으로 위험을 관리해야 할 무기는 고위험으로, 그 외의 AI 무기는 저위험으로 분류하여 차등화된 관리를 실시*
* 저위험 AI 무기는 법적·정책적 심사와 기술적 심사를 면제하고 사업 수행 담당자의 자체 점검을 중심으로 위험관리를 진행


# Qdarnt

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams, SparseVectorParams

# client = QdrantClient(":memory:") → 인메모리 벡터 저장소 생성.
client = QdrantClient(host="localhost", port=6333) # docker를 이용해 사용. 따라서 포트번호가 필요함.

collection_name = "RAG_Example(RAG_strategies)"

dense_embedding = OllamaEmbeddings(model="bge-m3")
sparse_embedding = FastEmbedSparse(model_name="Qdrant/bm25")

qdrant = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=dense_embedding,
    sparse_embedding=sparse_embedding,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse"
)


In [3]:
retriever = qdrant.as_retriever(search_kwargs={"k": 3})

In [4]:
response = retriever.invoke("미드저니")
response

[Document(metadata={'source': 'docling_outputs\\SPRi AI Brief_8월호_산업동향_F.md', '_id': '99fab946-a7a5-44fc-a827-38cba5e9b3ca', '_collection_name': 'RAG_Example(RAG_strategies)'}, page_content="- n AI 이미지 생성 플랫폼 미드저니(Midjourney)가 2025년 6월 19일 비디오 생성 모델 'V1'을 출시\n- V1은 이미지를 동영상으로 변환하는 모델로, 미드저니 플랫폼에서 제작된 이미지나 외부 이미지를 바탕으로 동영상을 생성하며, '자동' 설정 시에는 모션 프롬프트가 자동으로 생성되고 '수동' 설정을 선택하면 사용자 지시에 따라 사물이나 장면의 움직임을 생성\n- 사용자는 움직임 강도를 피사체와 카메라가 모두 움직이는 '하이 모션(High Motion)'과 카메라는 거의 고정되어 있고 피사체가 천천히 움직이도록 연출 '로우 모션(Low Motion)' 중에서 선택 가능\n- 웹 전용 모델인 V1은 1회 동영상 생성 작업으로 5초 길이의 동영상 4개를 제작하며, 생성된 동영상은 한 번에 4초씩 최대 4회까지 영상 길이를 확장할 수 있도록 허용"),
 Document(metadata={'source': 'docling_outputs\\SPRi AI Brief_8월호_산업동향_F.md', '_id': '3b53a337-de9e-47b0-a2f4-eaedb4519aee', '_collection_name': 'RAG_Example(RAG_strategies)'}, page_content="<!-- image -->\n\n## 기업 ･ 산업\n\n<!-- image -->\n\n## 미드저니, 첫 번째 비디오 생성 AI 모델 'V1' 출시\n\n## KEY Contents\n\n- n 미드저니가 1회 작업으로 5초 길이의 동영상 4개를 제작할 수 있는 비디오 생성 모델 'V1'을 출시하고 여타 비디오 생성 모델보다